# 06 - First-Level Analysis

In last week's notebook we had a look on how to inspect the quality of our data and preprocessing results. Once the data has been preprocessed we can start with the actual statistical analysis. Recall from the lecture that the first step of the analysis takes place on the subject level. That is, each subject's data will be modelled separately. This is also known as **First Level Analysis**. In this notebook, we will learn how to perform such analysis using Python with the help of the Nilearn's GLM submodule.



#### Questions

- How to make statistical inference on a single-subject level?


#### Objective

- Learn to create statistical maps using a mass univariate approach

## Install dependencies and data

In [ ]:
!pip install nilearn

For illustration purposes, we will use a dataset that comes with Nilearn (check out the documentation of the [```datasets```](https://nilearn.github.io/stable/modules/datasets.html) submodule for other available datasets).

The dataset includes the functional data of a single subject who completed the tasks described in [Pinel et al. (2007)](https://doi.org/10.1186/1471-2202-8-91). In their study, participants had to perform basic operations such as pressing a button with the left or right hand, reading and listening to short sentences or viewing horizontal and vertical checkerboards.

In [ ]:
from nilearn.datasets import func
data = func.fetch_localizer_first_level()

Below are the files that come with the Nilearn dataset. It compromises a preprocessed functional image (normalized to the MNI template) and a table (```.tsv```) specifying the experimental paradigm

In [ ]:
! ls /root/nilearn_data/localizer_first_level/localizer_first_level

As usual, we first load in the data, check the shape of it and plot the image

In [ ]:
import nibabel as nib

func = nib.load("/root/nilearn_data/localizer_first_level/localizer_first_level/sub-12069_task-localizer_space-MNI305.nii.gz")
print(func.shape)

In [ ]:
from nilearn import plotting, image

plotting.view_img(image.mean_img(func))

## Defining the experiment

The BIDS standard specifies that for each run and for each subject the dataset has to include a file specifying the timing and other properties of events that took place in the respective run - the ```events.tsv``` file (see the [BIDS](https://bids-specification.readthedocs.io/en/stable/04-modality-specific-files/05-task-events.html) documentation).

This file always has to include information regarding the time of the **onset** of the stimulus/condition, and its **duration**. It also includes further information regarding the experimental paradigm (such as **trial type**)

Let's have a look at the ```.tsv``` file of our dataset:

In [ ]:
import pandas as pd

events = pd.read_table("/root/nilearn_data/localizer_first_level/localizer_first_level/sub-12069_task-localizer_events.tsv")

# have a quick look at the data
print(events.head(5))
print(events.tail(5))

The experiment paradigm of our sample datasets includes 10 different conditions:

In [ ]:
print(events.trial_type.unique())

Thanks to the ```.tsv``` file we know when and for how long a given condition takes place in the experiment. This way, we can relate it to the timecourse of our fMRI data - using a general linear model (GLM)!


### Specifying the Model

To predict the voxels timecourses we will specify a general linear model using Nilearn's ```FirstLevelModel``` class:

In [ ]:
from nilearn.glm.first_level import FirstLevelModel

In [ ]:
help(FirstLevelModel)

As can be seen in the documentation, we need to provide the repition time (```t_r```) of our functional scans, i.e., the time it takes to scan one volume. This parameter is important as it implies at which point in time we have sampled the voxels responses.

Normally in a BIDS dataset, the repition time is specified in a ```.json``` file or included in the header. For this dataset, however, we have to set the repitition time manually (which can be found in Pinel et al. (2007)).

In [ ]:
tr = 2.4

Using the repition time, we can now setup our linear model using the ```FirstLevelModel``` class. In this step, we specify which hemodynamic reponse function (HRF) to use. Here, we will use SPM's default function, but we could also include further time derivatives as regressors (see documentation). Also, we will set a high pass filter (removing signals below that threshold) und use the default drift model (to account for signal drift in the course of the experiment).

### Fitting the model to the data

Now that we have specified the model, we can fit our data to it. In this step, we have to provide our events table. This way, the HRF and the timings of the respective conditions are [convolved](https://en.wikipedia.org/wiki/Convolution), resulting in an **ideal time-series** that represents a given voxel's time-series *if* that voxel is responsive to a given condition. Importantly, we are using a mass univariate approach, meaning that we want to model *every* voxel.


Note that in this step, we could also include the motion parameters as further regressors (see the confounds parameter of ```FirstLevelModel```) but they are not included in this dataset. For a tutorial where motion parameters have been included check the [References & Resources](#References-&-resources) section

In [ ]:
first_level = first_level.fit(image.smooth_img(func, 4), events = events)

The ```.fit``` method yields both the beta maps (i.e., the regression weight for each predictor) and the design matrix - the statistical model of the data. It contains the ideal time-series for each experimental condition (the predictor variables) as well as some further nuisance regressors (note that the motion parameters would have been included as further predictors).

*Note that since the data has not been smoothed during preprocessing, we are apply smoothing with a kernel width of 4 using Nilearn's ```smooth_img``` function, increasing statistical power*

We can visualize the design matrix using Nilearn (it's considered to inspect the design matrix before continuing with the analysis):

In [ ]:
design_matrix = first_level.design_matrices_[0]
plotting.plot_design_matrix(design_matrix)

In [ ]:
print(design_matrix.shape)

As you can see, the design matrix consists of 16 columns, representing the predictors (10 experimental conditions). The rows indicate the observations, i.e., points in the time-series. Each column consists of the ideal time-series for the respective conditions (go back to the events file and see if it makes sense to you)

Let's plot the first column (representing the ideal time-series for the audio computation time-series):

In [ ]:
import matplotlib.pyplot as plt

plt.plot(design_matrix["audio_computation"])

### Contrasts: comparing experimental conditions


Now, if we want to know which voxels respond more/less/differently to a certain condition than to another condition (or baseline), we have to compute **contrasts** - the difference in beta estimates between conditions.

To calculate a contrast, we have to assign weights to each Beta (i.e., each predictor -> design matrix column).

In the following we want to check which voxels show more/less activity in the sentence listening condition compared to the baseline. Furthermore, we want to know which voxels are more/less active when pressing a button with the right hand compared to pressing it with the left hand. Finally, we want to compute a contrast for all conditions involving predominantly visual operations.

In [ ]:
import numpy as np

conditions = {
  "sentence_listening > baseline":  np.array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  "right > left":                   np.array([0, 0, 0, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0, 0, 0, 0]),
  "avg_visual":                     np.array([0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0])
}

We have to assign a weight to each predictor of our design matrix. Thus the array to specify a contrast has a length of 16.

In [ ]:
for id, val in conditions.items():
  plotting.plot_contrast_matrix(val, design_matrix=design_matrix)

Above, we defined the contrasts manually (i.e., setting the weights ourselves). However, Nilearn also allows us to define simple contrasts with verbal labels (i.e., the design matrix column labels):

In [ ]:
plotting.plot_contrast_matrix("visual_left_hand_button_press - visual_right_hand_button_press",
                              design_matrix=design_matrix)

Now that we have defined the contrasts of interest, we can calculate the respective statistics. To investigate statistical significance we will calculate a t-statistic for each voxel and directly convert it into z-scale (we will use the ```conditions``` dictionary we defined earlier but you could also use verbal labels as in the cell above):

In [ ]:
z_map_right_left = first_level.compute_contrast(conditions["right > left"],
                                                output_type="z_score")

Next, we will display the statistical map on top of the MNI template (this is the default background image of the ```plot_stat_map``` function but we could also use the subject's anatonomy, see the ```bg_img``` parameter). Using the treshold parameter, we also specify that only voxels with an absolute z-value larger than 3 will be displayed (this is for illustration purposes only and no proper statistical inference).

In [ ]:
plotting.plot_stat_map(z_map_right_left,
                       display_mode="z",
                       cut_coords=5,
                       threshold = 3.0,
                       title="right_hand > left_hand", symmetric_cbar=True)

As expected, voxels in the motor/somatosensory cortex of the left hemisphere are more active when pressing a button with the right hand. To recap the theory of contrasts: For right>left contrast, we weigh the Betas of the right and left button press condition with 1 and -1, respectively. Consider for example a voxel in the left M1: Say for this voxel, our model estimated a β of 4 for the right button press condition (i.e., high activity) and a β of 0.1 for the left button press condition (almost no activity): We now weigh these β with our [1, -1] contrast (and 0 for all other conditions). That is: $$ 4*1 + 0.1 * -1 = 3.9  $$   

So for this specific voxel in the left M1 we would end up with z-scaled value of 3.9 (i.e., a rather high value -> yellowish dot in the statistical map). This is done for all voxels in the brain (mass univariate approach) and the end result is the statistical map we see above.

Importantly, you can also use such contrasts as a **sanity check**: Think about contrasts between conditions in your experiment where you are certain that a specific region shows a increased BOLD response (e.g. activity in the left motor cortex for a right-hand button press or V1 activity for eyes open vs. eyes closed). If the results don't match you expectations, chances are that something went wrong and you would have to double check your preprocessing pipeline.


#### Correcting for multiple comparisons

So far we haven't tested the voxels' responses for significance yet. For this, we will import another Nilearn function.

A popular way of correcting for multiple comparisons in the context of fMRI data is the [false discovery rate](https://en.wikipedia.org/wiki/False_discovery_rate) which is the expected proportion of false discoveries among detections - again, check the lecture slides or the [Nilearn documentation](https://nilearn.github.io/stable/glm/glm_intro.html#multiple-comparisons) for more information on this. Also, have a look at this nice [notebook on  multiple comparison correction](https://lukas-snoek.com/NI-edu/fMRI-introduction/week_6/MCC.html) created by [Lukas Snoek](https://lukas-snoek.com/) from the University of Amsterdam.

In [ ]:
from nilearn.glm.thresholding import threshold_stats_img
_, threshold = threshold_stats_img(z_map_right_left, alpha=.05, height_control='fdr')

plotting.plot_stat_map(z_map_right_left,
                       display_mode="z",
                       cut_coords=5,
                       threshold = threshold,
                       title="right_hand > left_hand (fdr corrected; p < 0.05)")

In line 2, ```_,``` is used to indicate that the first element of the tuple can be discarded.




Now we can apply these steps to all of our defined contrasts using a ```for``` loop:


In [ ]:
for id, val in conditions.items():

  z_map=first_level.compute_contrast(val, output_type="z_score")
  _, threshold = threshold_stats_img(z_map, alpha=.05, height_control='fdr')

  plotting.plot_stat_map(z_map,
                         display_mode="z",
                         threshold=threshold,
                         cut_coords=range(-20, 70, 10),
                         title=id)

## Reporting

### Automatic reporting using Nilearn

Another way of checking the results of the first level analysis is to inspect the significant clusters of voxels. For this job, Nilearn provides the ```get_clusters_table``` function which will list the clusters with the most activity.

We will set a treshold of 10, so that only clusters with more than 10 voxels will be included in our table (note that this is an arbitrary value and only done here for illustration purposes).

Let's get the cluster statistics for the right hand > left hand contrast:

In [ ]:
from nilearn.glm.thresholding import threshold_stats_img
from nilearn.reporting import get_clusters_table

_, threshold = threshold_stats_img(z_map_right_left, alpha=.05, height_control='fdr')

cluster_table = get_clusters_table(z_map_right_left, stat_threshold=threshold,
                                   cluster_threshold=10)

print(cluster_table)



Another nice feature of Nilearn is the ```make_glm_report``` function which outputs a HTML report for all important aspects of a fitted GLM:

In [ ]:
from nilearn.reporting import make_glm_report
my_glm_report = make_glm_report(first_level, contrasts = conditions,
                                height_control = "fdr", alpha=.05)

my_glm_report

We can inspect the report directly in the notebook or save it as a ```.html``` file using the ```save_as_html()``` method

In [ ]:
my_glm_report.save_as_html("my_report.html")

### Adding region labels

Another neat way to automatically generate reports of our results is implemented in the Python package [```atlasreader```](https://github.com/miykael/atlasreader). A cool thing about this tool is that it does not only create nice figures but also adds atlas labels for the cluster peaks.

In this notebook, we only want to extract a cluster table containing the respective atlabs label - thus we import the ```get_statmap_info``` (however, do check the Githup repository of atlasreader to refer to the usage of its main function ```create_output```):

In [ ]:
!pip install atlasreader

In [ ]:
from atlasreader import get_statmap_info

In [ ]:
cluster_info, peak_info = get_statmap_info(z_map_right_left,
                                           cluster_extent = 10)

In [ ]:
cluster_info.head(10)

---

What we did not cover in this notebook are ways to assess the quality of the model fit. For this, you can inspect residuals of the model as well as extract the predicted time series (you need to set ```minimize_memory``` in when initializing the ```FirstLevelModel``` to ```False``` to do so). See [here](https://nilearn.github.io/dev/auto_examples/04_glm_first_level/plot_predictions_residuals.html#sphx-glr-auto-examples-04-glm-first-level-plot-predictions-residuals-py for a tutorial from the Nilearn user guide

## References & Resources

**Dataset**

Pinel, P., Thirion, B., Meriaux, S. et al. Fast reproducible identification and large-scale databasing of individual functional cognitive networks. *BMC Neuroscience*, *91* (2007). https://doi.org/10.1186/1471-2202-8-91

Notter M. P., Gale D., Herholz P., Markello R. D., Notter-Bielser M.-L., & Whitaker K. (2019). AtlasReader: A Python package to generate coordinate tables, region labels, and informative figures from statistical MRI images. *Journal of Open Source Software, 4(34), 1257*, [https://doi.org/10.21105/joss.01257](https://doi.org/10.21105/joss.01257)


---


**Tutorials covering First-Level analysis in Python:**


[Nilearn User Guide on First-Level models](https://nilearn.github.io/stable/glm/first_level_model.html)


[Nilearn statistical analysis tutorial](https://github.com/miykael/workshop_pybrain/blob/master/workshop/notebooks/04b_statistical_analyses_MRI.ipynb) & corresponding [Youtube tutorial](https://youtu.be/4FVGn8vodkc?t=15105)
- includes both first and second level analysis
- covers further concepts, such as inclusion of motion parameters, F-tests or model evaluation

[Nilearn documentation: GLM First Level](https://nilearn.github.io/auto_examples/04_glm_first_level/plot_first_level_details.html#sphx-glr-auto-examples-04-glm-first-level-plot-first-level-details-py)
- same dataset as in this notebook, but more in-depth
- data masking, changing the HRF function, adjusting the drift model

Below, there also two more in-depth tutorials covering the First-Level analysis within Python, however, with "manual" model specification and fitting (i.e., without the help of Nilearn).

https://lukas-snoek.com/NI-edu/fMRI-introduction/week_2/glm_part1_estimation.html

https://lukas-snoek.com/NI-edu/fMRI-introduction/week_3/glm_part2_inference.html
